In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
# Import db related librarys
import psycopg2
import sys
# import boto3
import os
import sqlalchemy
import json
import csv
from getpass import getpass


In [2]:
# Configure connection parameters to AWS RDS
def connect_to_db():
    
    connection = None
    #enter_password = getpass('Enter database password')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(
            host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
            port = 5432,
            user = 'postgres',
            password = getpass('Enter database password'),
            database = 'postgres'
            )
        print('Connection successful')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return connection


In [3]:
# Call the connect_to_db to connect to database
connection = connect_to_db()

Connecting to the PostgreSQL database...
Enter database password········
Connection successful


In [4]:
# Create cursor to perform database operations
cursor = connection.cursor()

# If curser object = 0, then connection was successfully established
cursor

<cursor object at 0x000001774740A668; closed: 0>

In [5]:
# Query the location_table from AWS database and store in dataframe - verify there is data in the table.
sql = """
SELECT * FROM location_table
"""

location_table_from_db = pd.read_sql(sql, con=connection)

In [6]:
# Display the location_table queried from database
location_table_from_db

,country_code,latitude,longitude,broad_bean_origin_country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
242,ZM,-13.133897,27.849332,Zambia
243,ZW,-19.015438,29.154857,Zimbabwe
244,HI,19.898682,-155.665857,Hawaii
245,ZZ,0.000000,0.000000,Unknown


In [7]:
# Query the clean_flavours_table from AWS database and store in dataframe.
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [8]:
clean_flavors_from_db

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1955,Zotter,Peru,647,2011,70.0,Austria,3.75,missing,Peru,"4- B,Sw,C,Sa","creamy, fatty, mild nutty",South America
1956,Zotter,Congo,749,2011,65.0,Austria,3.00,Forastero,Republic of Congo,"4- B,Sw,C,Sa","dairy, salt, caramel",Africa
1957,Zotter,Kerala State,749,2011,65.0,Austria,3.50,Forastero,India,"4- B,Sw,C,Sa","creamy, masculine, earthy",Asia
1958,Zotter,Kerala State,781,2011,62.0,Austria,3.25,missing,India,"4- B,Sw,C,Sa","oily, subdued, caramel, salt",Asia


In [9]:
# Join the 2 tables together on broad_bean_origin_country, select the columns you want to view in df

sql = """ SELECT clean_flavors_table .*, location_table.country_code,location_table.longitude, location_table.latitude
FROM clean_flavors_table
LEFT JOIN location_table
ON clean_flavors_table.broad_bean_origin_country = location_table.broad_bean_origin_country;
"""

# Store the joined tables in dataframe
df_chocolate = pd.read_sql(sql, con=connection)

# View the new dataframe combined from two sql tables
df_chocolate.head(10)

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent,country_code,longitude,latitude
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa,ST,6.613081,0.186360
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa,TG,0.824782,8.619543
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa,TG,0.824782,8.619543
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa,TG,0.824782,8.619543
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America,PE,-75.015152,-9.189967
5,A. Morin,Carenero,1315,2014,70.0,France,2.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
6,A. Morin,Cuba,1315,2014,70.0,France,3.50,missing,Cuba,"4- B,S,C,L","sliglty dry, papaya",Caribbean,CU,-77.781167,21.521757
7,A. Morin,Sur del Lago,1315,2014,70.0,France,3.50,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
8,A. Morin,Puerto Cabello,1319,2014,70.0,France,3.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
9,A. Morin,Pablino,1319,2014,70.0,France,4.00,missing,Peru,"4- B,S,C,L","delicate, hazelnut, brownie",South America,PE,-75.015152,-9.189967


In [10]:
# Close connection to RDS
cursor.close()

In [11]:
# Check that connection is closed where object returned = -1
cursor

<cursor object at 0x000001774740A668; closed: -1>

# Machine Learning 



In [12]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 0
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [13]:
df_chocolate['rating'] = df_chocolate['rating'].apply(bin_ratings)


In [14]:
def to_string(value):
    other = f"_{value}_"
    return other

In [15]:
def reduce_count_vals(df, colname, threshold):
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")

In [16]:
df_chocolate['review_date']= df_chocolate['review_date'].apply(to_string)
reduce_count_vals(df_chocolate, 'review_date', 100)

## Neural Network!

In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_chocolate= df_chocolate.drop(columns=["company_location","REF","bean_origin_or_bar_name","most_memorable_characteristics","latitude","longitude","country_code"], axis=1)
df_chocolate.head()

,company,review_date,cocoa_percent,rating,bean_type,broad_bean_origin_country,ingredients,continent
0,A. Morin,_2016_,63.0,1,missing,Sao Tome & Principe,"4- B,S,C,L",Africa
1,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
2,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
3,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
4,A. Morin,_2015_,70.0,0,missing,Peru,"4- B,S,C,L",South America


In [18]:
df_chocolate.describe()

,cocoa_percent,rating
count,1960.000000,1960.000000
mean,71.760204,0.166327
std,6.657419,0.372469
min,42.000000,0.000000
25%,70.000000,0.000000
50%,70.000000,0.000000
75%,75.000000,0.000000
max,100.000000,1.000000


In [19]:

# continent_names_drop = df_chocolate[(df_chocolate.continent.str.contains("Unknown"))].index
# or (df_chocolate.continent.str.contains("Other"))]                                  
# # continent_names_drop
# df_chocolate = df_chocolate.loc[df_chocolate['continent']!= "Unknown" , :]
# # df_chocolate = df_chocolate.loc[df_chocolate['continent']!= "Other" , :]
# df_chocolate.describe()

In [20]:
# df_chocolate = df_chocolate.loc[df_chocolate['continent']!= "Other" , :]
# df_chocolate.describe()

In [21]:
# Generate our categorical variable lists
chocolate_cat = df_chocolate.dtypes[df_chocolate.dtypes == 'object'].index.tolist()
chocolate_cat

['company',
 'review_date',
 'bean_type',
 'broad_bean_origin_country',
 'ingredients',
 'continent']

In [22]:


# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_chocolate[chocolate_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(chocolate_cat)
encode_df.head()



,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,company_Alexandre,company_Altus aka Cao Artisan,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Merge one-hot encoded features and drop the originals
df_chocolate = df_chocolate.merge(encode_df, left_index=True, right_index=True).drop(chocolate_cat, 1)
df_chocolate.head()

C:\Users\bjile\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,cocoa_percent,rating,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,63.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Split our preprocessed data into our features and target arrays
y = df_chocolate["rating"]
X = df_chocolate.drop(["rating"],1) 

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.4,train_size=0.6 , random_state=1, stratify=y)

C:\Users\bjile\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
len(X_train_scaled[0])

523

In [27]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)


# Random Forest

In [28]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.889


In [29]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.889030612244898

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,582,72
Actual high rated chocolate,15,115


In [31]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       654
           1       0.61      0.88      0.73       130

    accuracy                           0.89       784
   macro avg       0.79      0.89      0.83       784
weighted avg       0.92      0.89      0.90       784



In [32]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_



In [33]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.13824753601548873, 'cocoa_percent'),
 (0.02373017822374848, 'bean_type_missing'),
 (0.021990397714143725, 'review_date__2009_'),
 (0.02161364735131794, 'ingredients_3- B,S,C'),
 (0.021525354693058437, 'review_date__2014_'),
 (0.02002190386669965, 'review_date__2015_'),
 (0.018368825749625974, 'broad_bean_origin_country_Ecuador'),
 (0.01806808592255418, 'company_Soma'),
 (0.017706380430141746, 'continent_South America'),
 (0.017443373010099197, 'review_date__2012_'),
 (0.01729836167341514, 'broad_bean_origin_country_Venezuela'),
 (0.01647144553538531, 'ingredients_2- B,S'),
 (0.016044653307256622, 'ingredients_4- B,S,C,V'),
 (0.015883561615925624, 'ingredients_Unknown'),
 (0.015759844685435696, 'review_date__2016_'),
 (0.015367596274617885, 'bean_type_Trinitario'),
 (0.014700573611966843, 'continent_Central America'),
 (0.014015298274453162, 'review_date__2013_'),
 (0.013820971019537382, 'bean_type_Criollo'),
 (0.012788744420297083, 'broad_bean_origin_country_Peru'),
 (0.01272896403

# Support Vector Machine

In [34]:
# Create the SVM model
svm = SVC(kernel='rbf')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.832


In [35]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8316326530612245

In [36]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,640,14
Actual low_risk,118,12


In [37]:
# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.98      0.91       654
           1       0.46      0.09      0.15       130

    accuracy                           0.83       784
   macro avg       0.65      0.54      0.53       784
weighted avg       0.78      0.83      0.78       784



# Deep Learning Model

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
hidden_nodes_layer5 = 16
hidden_nodes_layer6 = 8
hidden_nodes_layer7 = 4


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu'))

# other hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer4, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer5, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer6, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer7, activation = 'relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               134144    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3

In [39]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [40]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

In [41]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [42]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100, callbacks=[cp_callback])

Epoch 1/100
37/37 [==============================] - 1s 2ms/step - loss: 0.4533 - accuracy: 0.8342
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.3394 - accuracy: 0.8333
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.2557 - accuracy: 0.8767
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1957 - accuracy: 0.9175
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1715 - accuracy: 0.9294

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1559 - accuracy: 0.9345
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1312 - accuracy: 0.9498
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1099 - accuracy: 0.9507
Epoch 9/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1039 - accuracy: 0.9524
Epoch 10/100
37/37 [==============================]

37/37 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 0.9881
Epoch 75/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0179 - accuracy: 0.9898

Epoch 00075: saving model to checkpoints\weights.75.hdf5
Epoch 76/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 0.9889
Epoch 77/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0197 - accuracy: 0.9872
Epoch 78/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 0.9872
Epoch 79/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 0.9881
Epoch 80/100
37/37 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 0.9881

Epoch 00080: saving model to checkpoints\weights.80.hdf5
Epoch 81/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0174 - accuracy: 0.9889
Epoch 82/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0171 - accuracy: 0.99

In [43]:


# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



25/25 - 0s - loss: 2.7568 - accuracy: 0.7730
Loss: 2.7567806243896484, Accuracy: 0.7729591727256775


In [44]:
nn.save("Chocolate_Ratings_ML.h5")